In [3]:
import numpy as np
import cv2
import math 
from shapely.geometry import Polygon
import matplotlib.pyplot as plt
from sklearn.model_selection import ParameterGrid
from matplotlib import colors
from scipy.signal import argrelextrema
import pandas as pd
import csv
import os
from scipy.ndimage import gaussian_filter1d
import re

In [29]:
image_folder = './testing_slices'

# Iterate through case folders and rename slices to the appropriate format
match_folder = re.compile(r"match[ _]?(\d+)", re.IGNORECASE)
slice_file = re.compile(r"([a-zA-Z]\d+)[ _]+(h&e|melan|sox10)[ _]+ROI[ _]+(\d+).(png|tif)")
for dirpath, dirnames, filenames in os.walk(image_folder):
    for dirname in dirnames:
        match = match_folder.fullmatch(dirname)
        if match:
            match_number = int(match.group(1))
            match_path = os.path.join(dirpath, dirname)
            case_number = str(int(os.path.basename(dirpath)[-3:]))
            for slic in os.listdir(match_path):
                slice_match = slice_file.fullmatch(slic)
                if slice_match:
                    case_id, stain, snum, extension = slice_match.groups()
                    new_filename = f"case_{case_number}_match_{match_number}_{stain}.{extension}"
                    old_path = os.path.join(match_path, slic)
                    new_path = os.path.join(match_path, new_filename)
                    os.rename(old_path, new_path)
                    print(f"Renaming: {slic} -> {new_filename}")

# Rename annotated mask files in testing slices folder
for filename in os.listdir(image_folder):
    match = re.match(r"(h&e|melan|sox10).*?case(\d+)_match(\d+)\.(png|tif)", filename)
    if match:
        stain, case_number, match_number, extension = match.groups()
        new_filename = f"case_{case_number}_match_{match_number}_{stain}.{extension}"
        old_path = os.path.join(image_folder, filename)
        new_path = os.path.join(image_folder, new_filename)
        os.rename(old_path, new_path)
        print(f"Renaming: {filename} -> {new_filename}")

Renaming: h2114181  h&e_ROI_4.tif -> case_28_match_1_h&e.tif
Renaming: h2114181 melan_ROI_2.tif -> case_28_match_1_melan.tif
Renaming: h2114181 sox10_ROI_1.tif -> case_28_match_1_sox10.tif
Renaming: h2114183 h&e_ROI_3.tif -> case_30_match_1_h&e.tif
Renaming: h2114183 melan_ROI_1.tif -> case_30_match_1_melan.tif
Renaming: h2114183 sox10_ROI_1.tif -> case_30_match_1_sox10.tif
Renaming: h2114185 h&e_ROI_3.tif -> case_32_match_1_h&e.tif
Renaming: h2114185 melan_ROI_2.tif -> case_32_match_1_melan.tif
Renaming: h2114185 sox10_ROI_1.tif -> case_32_match_1_sox10.tif
Renaming: h2114186 h&e_ROI_2.tif -> case_33_match_1_h&e.tif
Renaming: h2114186 melan_ROI_2.tif -> case_33_match_1_melan.tif
Renaming: h2114186 sox10_ROI_1.tif -> case_33_match_1_sox10.tif
Renaming: h2114187 h&e_ROI_1.tif -> case_34_match_1_h&e.tif
Renaming: h2114187 melan_ROI_1.tif -> case_34_match_1_melan.tif
Renaming: h2114187 sox10_ROI_1.tif -> case_34_match_1_sox10.tif
Renaming: h2114188 h&e_ROI_1.tif -> case_35_match_1_h&e.tif

In [23]:
# End code to execute in patching code
for dirpath, dirnames, filenames in os.walk(image_folder):
    for filename in filenames:
        match = re.match(r"case_(\d+)_match_(\d+)_(h&e|melan|sox10).*\.(png|tif)", filename)
        if match:
            case_number, match_number, scan_type, extension = match.groups()
            case_number, match_number = int(case_number), int(match_number)  # Convert to int for sorting
            
            if case_number not in cases:
                cases[case_number] = {}
    
            if match_number not in cases[case_number]:
                cases[case_number][match_number] = {}
    
            if scan_type not in cases[case_number][match_number]:
                cases[case_number][match_number][scan_type] = {"mask": "None", "slice": "None"}
    
            if extension == "png":
                cases[case_number][match_number][scan_type]["mask"] = os.path.join(image_folder, filename)
                
            elif extension == "tif":
                cases[case_number][match_number][scan_type]["slice"] = os.path.join(image_folder, filename)

# Build list of tuples (mask, slice), ensuring both exist
matched_pairs = []
masks = []
for case in cases.values():
    for match in case.values():
        for scan in match.values():
            if 'mask' in scan:
                matched_pairs.append((scan['mask'], scan['slice']))


# delete any patches that are already in the directory
empty_folder()

# Run patching and exporting over list of tuples
for i, m in enumerate(matched_pairs, 1):
    print(m)
    image1 = cv2.imread(m[0])
    image1_gray = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
    _,whole_thresh1=cv2.threshold(image1_gray,240,255,cv2.THRESH_BINARY_INV)
    if mask_smoothing == True:
        # Compute the smoothed mask.
        smoothed_binary = compute_smoothed_mask(whole_thresh1)
        # Use the smoothed mask in your patching/export pipeline.
        metrics, images = patching_export(smoothed_binary, m[0], m[1])
    else:
        metrics, images = patching_export(whole_thresh1, m[0], m[1])  # (binary, slice_name, slice_details) aka (binary image, epithelium annotation png, tif)
    print(metrics)

NameError: name 'cases' is not defined